# TP1: PageRank

Nome: *escreva seu nome aqui*

Matrícula: *escreva sua matrícula aqui*

* Data de entrega: até 23:59 do dia 17/05/2018

* Questões podem ser discutidas entre até três alunos. Nomes dos colegas precisam ser listados. Contudo, a escrita do código e a submissão deve ser feita individualmente.

* Todo material consultado na Internet deve ser referenciado (incluir URL).

* Submissão deve ser feita em formato de ipython notebook (extensão .ipynb) através do Moodle.

Este trabalho está dividido em cinco partes:
 * **Parte 0**: Esta parte não vale nota, mas é fundamental para entender o que se pede a seguir.
 * **Parte 1**: Pagerank sem saltos aleatórios em grafo pequeno
 * **Parte 2**: Pagerank (com saltos aleatórios) em grafo pequeno
 * **Parte 3**: Manipulação de matrizes esparsas
 * **Parte 4**: Pagerank sparso em grafo grande

## Parte 0: Revisão de conceitos

I. O **primeiro autovetor** (isto é, o autovetor associado ao maior autovalor em módulo) pode ser calculado rapidamente através do método da potência, desde que o *gap* entre o maior e o segundo maior autovalor (em módulo) seja grande. Uma implementação simples do método da potência é mostrada a seguir.

In [143]:
def powerMethod(A, niter=10):
    n = len(A)
    w = np.ones((n,1))/n
    for i in range(niter):
        w = A.dot(w)        
    return w

II. Dado um grafo $G=(V,E)$, podemos obter uma **matriz de probabilidade de transição** $P$ dividindo-se cada linha de $A$ pela soma dos elementos da linha. Seja $D = A \times \mathbf{1}$ a matriz diagonal contendo a soma das linhas de $A$. Temos que

$$
P = D^{-1} \times A.
$$

III. A matriz de probabilidade de transição $P$ de certos grafos direcionados satisfaz

$$
v^\top P = v^\top \textrm{ou $P^\top v = v$},
$$

onde $v$ é o primeiro autovetor de $P^\top$. A equação da direita é mais fácil de ser utilizada, pois ela tem a forma canônica $Ax=b$. Já a equação da direita é mais fácil de ser interpretada. Para todo $j=1,\ldots,n$,

$$
\sum_{i=1} v_i P_{ij} = v_j \\
\sum_{i=1} v_i \frac{A_{ij}}{D_{ii}} = v_j \\
\sum_{i:(i,j) \in E} v_i \frac{1}{D_{ii}} = v_j
$$

IV. Assuma que $v$ seja normalizado de forma que $\sum_j v_j = 1$. O PageRank (sem saltos) de um vértice $j$ é dado por $v_j$, onde $v$ é o primeiro autovetor de $P^\top$. Esta é uma maneira de medir sua relevância. A intuição da Equação $\sum_{i:(i,j) \in E} v_i /D_{ii} = v_j$ é que a relevância de $j$ é a soma das relevâncias dos vértices $i$ que apontam para $j$ normalizados pelos seus respectivos graus de saída.

## Parte 1: Pagerank sem saltos aleatórios em grafo pequeno

Considere o grafo a seguir composto por $n=4$ vértices e $m=8$ arestas. 
<img src="images/directedgraph.png"/>

Certifique-se de que encontrou as $m=8$ arestas.

**1.1** Crie um numpy array $A$ contendo a matriz de adjacência.

In [138]:
import numpy as np
A = np.array()

**1.2** Escreva uma função matrizDeTransicao que recebe como entrada uma matriz $n \times n$ e retorna a matriz de probabilidade de transição. Imprima $P$, a matriz de transição para $A$.

In [139]:
def matrizDeTransicao(M):
    return 

P = matrizDeTransicao(A)
print(P)

[[0.         0.5        0.5        0.        ]
 [0.         0.         0.5        0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333 0.        ]]


**1.3** Use a função np.linalg.eig para calcular o primeiro autovetor de $P^\top$. Normalize o autovetor pela sua soma e imprima o resultado. (Observação: os elementos do autovetor serão retornados como números complexos, mas a parte imaginária será nula e pode ser ignorada.)

In [4]:
autoval,autovec = np.linalg.eig()
autovec =
print()

[0.12903226+0.j 0.19354839+0.j 0.29032258+0.j 0.38709677+0.j]


**1.4** Verifique que o método da potência aplicado a $P^\top$ retorna uma aproximação para o primeiro autovetor. 

In [5]:
powerMethod()

array([[0.12885802],
       [0.19331115],
       [0.29024402],
       [0.38758681]])

**1.5** Implemente uma função powerMethodEps(A, epsilon) que executa o método da potência até que a condição de convergência $\|w_{t} - w_{t-1}\| < \epsilon$ seja atingida. Para a matriz $P^\top$ com $\epsilon=10^{-5}$ quantas iterações foram realizadas? 

In [6]:
def powerMethodEps(A, eps=1e-5):
    n = len(A)
    w = np.ones((n,1))/n
    w_new = A @ w
    count = 1
    while :
        w = w_new
        w_new = A @ w
    print(count)
    
    return w

powerMethodEps(P.T)

20


array([[0.12903346],
       [0.19354849],
       [0.29032191],
       [0.38709614]])

## Parte II: Pagerank (com saltos aleatórios) em grafo pequeno

Agora iremos modificar a matriz A de forma a:
 * adicionar um novo vértice 4, e
 * adicionar uma aresta de 3 para 4.
 
Obviamente a matriz de probabilidade de transição não está definida para a nova matriz $A$. Vértices que não possuem arestas de saída (como o vértice 4) são conhecidos como *dangling nodes*. Para resolver este e outros problemas, incluiremos a possibilidade de realizar saltos aleatórios de um vértice para qualquer outro vértice.

Em particular, assume-se que com probabilidade $\alpha$, seguimos uma das arestas de saída em $A$ e, com probabilidade $1-\alpha$ realizamos um salto aleatório, isto é, transicionamos do vértice $v$ para um dos $n$ vértices do grafo (incluindo $v$) escolhido uniformemente. Quando não existem *dangling nodes*, a nova matriz de probabilidade de transição é dada por

$$
P = \alpha D^{-1} A + (1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n}
$$

Quando existem *dangling nodes*, a única possibilidade a partir desses nós é fazer saltos aleatórios. Mais precisamente, se $i$ é um vértice sem arestas de saída, desejamos que a $i$-ésima linha de $P$ seja o vetor $[1/n,\ldots,1/n]$. Uma forma de satisfazer essa definição é preencher com 1's as linhas de $A$ que correspondem aos *dangling nodes*. Uma desvantagem desta estratégia é que faz com que $A$ fique mais densa (mais elementos não-nulos).

Um valor típico usado para $\alpha$ é $0.85$.

**2.1** Crie um novo numpy array $A_\textrm{new}$ contendo o vértice 4 e a aresta (3,4).

In [7]:
A_new = 

**2.2** Crie uma função fixDangling(M) que retorna uma cópia modificada da matriz de adjacência M onde cada *dangling node* do grafo original possui arestas para todos os vértices do grafo. *Dica:* Você pode criar um vetor $d$ com os graus de saída e acessar as linhas de $M$ correpondentes aos *dangling nodes* por $M[d==0,:]$. Imprima a matriz $A_\textrm{fixed}$ retornada após chamar fixDangling para $A_\textrm{new}$.  

In [8]:
def fixDangling(M):
    M2 = M.copy()

    return M2
    
A_fixed = fixDangling(A_new)
A_fixed

array([[0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1]])

**2.3** Crie uma função matrizDeTransicao(M, alpha) que receba como parâmetro também a probabilidade $\alpha$ de não fazermos um salto aleatório. Você pode assumir que $M$ foi retornada por fixDanglig, logo, não possui *dangling nodes*. Imprima as matrizes:
 * $P_2$ obtida ao chamar matrizDeTransicao para os parâmetros $A$ e $\alpha=0.85$;
 * $P_\textrm{new}$ obtida ao chamar matrizDeTransicao para os parâmetros $A_\textrm{fixed}$ e $\alpha=0.85$.

In [9]:
def matrizDeTransicao(A, alpha=1.0):
    n = A.shape[0]
    d = A.sum(axis=1)
    return 

P_2 = matrizDeTransicao(A,0.85)
print(P_2)
P_new = matrizDeTransicao(A_fixed,0.85)
print(P_new)

[[0.0375     0.4625     0.4625     0.0375    ]
 [0.0375     0.0375     0.4625     0.4625    ]
 [0.0375     0.0375     0.0375     0.8875    ]
 [0.32083333 0.32083333 0.32083333 0.0375    ]]
[[0.03   0.455  0.455  0.03   0.03  ]
 [0.03   0.03   0.455  0.455  0.03  ]
 [0.03   0.03   0.03   0.88   0.03  ]
 [0.2425 0.2425 0.2425 0.03   0.2425]
 [0.2    0.2    0.2    0.2    0.2   ]]


**2.4** Imprima o resultado do método da potência com:
* $P_2^\top$ e $\epsilon=10^{-5}$
* $P_\textrm{new}^\top$ e $\epsilon=10^{-5}$.

In [10]:
pr_2 = powerMethodEps(P_2.T,eps=1e-5)
pr_new = powerMethodEps(,eps=1e-5)
print(pr_2)
print(pr_new)

16
14
[[0.14180823]
 [0.20207804]
 [0.2879621 ]
 [0.36815162]]
[[0.12190027]
 [0.17370876]
 [0.24753563]
 [0.33495506]
 [0.12190027]]


**2.5** Sejam $i_\max$ e $i_\min$ os índices dos vértices com maior e menor PageRank de $A_\textrm{fixed}$. Vamos verificar como a adição de um novo link pode ajudar a promover uma página web (vértice). Adicione uma aresta do vértice $i_\max$ para o vértice $i_\min$ (se já houver aresta, aumente de 1 para 2 o elemento da matriz de adjacência). Qual é o novo pagerank de $i_\min$?

In [11]:
ind_sorted = np.argsort(np.squeeze(pr_new))
imax = ind_sorted[-1]
imin = ind_sorted[0]

In [12]:
A_fixed2 = A_fixed.copy()

P_fixed2 = matrizDeTransicao(A_fixed2,0.85)
pr = powerMethodEps(P_fixed2.T,eps=1e-4)
print(pr)

13
[[0.1583561 ]
 [0.17021269]
 [0.24253671]
 [0.32596931]
 [0.10292519]]


## Parte 3: Manipulação de matrizes esparsas

*Parte do material foi traduzido de http://www.scipy-lectures.org/advanced/scipy_sparse/introduction.html*

Nesta parte do TP faremos algumas manipulações simples envolvendo matrizes esparsas.

Matrizes (densas) são:
* um objeto matemático
* uma estrutura de dados usada para armazenar um array de valores 2D

Features importantes:
* A memória é alocada de uma vez para todos os itens
* Provê acesso rápido aos seus itens

**Q:** Por que usar matrizes esparsas?

**R:** O uso da memória cresce quadraticamente para matrizes densas.

Matrizes esparsas e esquemas de armazenamento
* Uma matriz esparsa é uma matriz que é quase vazia
* Armazenar todos os elementos é um desperdício => é melhor armazenar apenas os elementos não-nulos
* Pense em **compressão**
* Prós: economia dramática de memória
* Contras: dependendo do esquema de armazenamento, o acesso aos itens pode não ser rápido

Aplicações típicas:
* Solução de equações diferenciais parciais (EDPs)
* Teoria dos grafos

**3.1** O formato COO (Coordinate Format) é tipicamente usado para inicializar matrizes esparsas. O construtor recebe como entrada um par (**data**, **indices**) onde data é uma lista contendo os elementos não-nulos da matriz e **indices** é um par (**ind_linhas**, **ind_colunas**), onde **ind_linhas** são os índices das linhas e **ind_colunas** são os índices das colunas.

Mostre como criar a matriz de adjacência $A_\textrm{sparse}$ usando o scipy.sparse.coo_matrix. 

In [123]:
import scipy.sparse

data = 
ind_linhas = 
ind_colunas = 

A_sparse = scipy.sparse.coo_matrix((data,(ind_linhas,ind_colunas)))

**3.2** Após criarmos uma matriz espasa do tipo COO, ela é tipicamente convertida para o formato CSR (Compressed Sparse Row) ou CSC (Compressed Sparse Column).

O código a seguir converte a matriz anterior para o formato CSC. Imprima uma mensagem que explique o que está armazenado em A_sparse.indices.

In [203]:
A_sparse = scipy.sparse.csc_matrix(A_sparse)
print(A_sparse.indices)
print('indices[i] corresponde a ...')

[3 0 3 0 1 3 1 2]
indices...


**3.3** Imprima uma mensagem que explique o que está armazendo em A_sparse.indptr.

In [204]:
print(A_sparse.indptr)
print('indptr[i] corresponde a ...')

[0 1 3 6 8]
indptr...


**3.4** Implemente o método somaDasLinhas que retorne a soma das linhas ao receber como parâmetros uma matriz esparsa do tipo CSC. Este método deve ter o comportamento semelhante ao de A_sparse.sum(axis=1).

In [205]:
A

array([[0, 1, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1],
       [1, 1, 1, 0]])

In [210]:
def somaDasLinhas(A):
    data = A.data
    indices = A.indices
    n = len(indices)
    soma = np.zeros(len(A.indptr)-1)
    for i in range(n):
        soma[indices[i]] += data[i] 
    return soma

print(somaDasLinhas(A_sparse),A_sparse.sum(axis=1))

[2. 2. 1. 3.] [[2]
 [2]
 [1]
 [3]]


**3.5** É um pouco estranho, mas o numpy considera matrix e array 2D como objetos diferentes. Uma forma de converter a matrix $M$ para array 2D é fazer np.asarray(M). Em alguns casos, $M$ é uma matriz linha ou uma matriz coluna e desejamos transformá-la em um array 1D. Para isto, podemos chamar M.A1. Corrija o código abaixo para somar o array $v$ e a soma das linhas de $A_\textrm{sparse}$.

In [137]:
v = np.array([0,0,0,0])
somaLinha = A_sparse.sum(axis=1)
v + somaLinha

matrix([[2, 2, 2, 2],
        [2, 2, 2, 2],
        [1, 1, 1, 1],
        [3, 3, 3, 3]])

## Parte 4: Pagerank esparso em grafos grandes

Nesta parte do TP vamos escrever novas versões dos métodos anteriores que façam uso de representações esparsas das matrizes. Desta forma, podemos trabalhar com grafos grandes sem utilizar muita memória. No entanto, é fundamental garantir que as representações esparsas sejam usadas durante todos os passos. Por exemplo, não podemos criar uma matriz diagonal $D$ não-esparsa em nenhum momento.

Para testar nossas implementações, vamos representar a matriz de adjacência do grafo $G$ com estruturas esparsas da biblioteca scipy.sparse.

In [219]:
import scipy.sparse
A_sparse = scipy.sparse.csc_matrix(A)

### Implementação do cálculo da matriz de transição

Nesta parte também iremos considerar saltos aleatórios. Contudo, se somássemos $A_\textrm{sparse}$ e $\frac{\mathbf{1}\mathbf{1}^\top}{n}$, teríamos uma matriz densa novamente. Por esta razão, o método matrizDeTransicaoEsparsa que você irá implementar irá apenas normalizar as linhas. Ele não deve receber o parâmetro $\alpha$.

**4.1** Implemente o método matrizDeTransicaoEsparsa(M) que retorna uma cópia de M, onde cada linha é normalizada por sua soma. Você pode assumir que a soma de cada linha é estritamente positiva. Salve  na variável $X$ o resultado retornado ao chamar a função para a matriz A_sparse.

In [220]:
def matrizDeTransicaoEsparsa(A_sparse):
    B = A_sparse.copy()
    d = B.sum(axis=1)
    B.data = B.data / 
    return B

In [221]:
P_sparse = matrizDeTransicaoEsparsa(A_sparse)
P_sparse.todense()

matrix([[0.        , 0.5       , 0.5       , 0.        ],
        [0.        , 0.        , 0.5       , 0.5       ],
        [0.        , 0.        , 0.        , 1.        ],
        [0.33333333, 0.33333333, 0.33333333, 0.        ]])

O método powerMethod fornecido no início TP pode ser aplicado a matrizes esparsas sem problemas, pois a única operação que ele realiza é a multiplicação de uma matriz por um vetor, que retorna um vetor. Portanto, não cria novas matrizes densas.

No entanto, iremos criar uma versão nova do método que em vez de executar o método da potência para matriz $M$ recebida como entrada, irá executá-lo para $\alpha M + (1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n}$. O truque para realizar este cálculo sem criar grandes matrizes densas é distribuir a multiplicação:

$$
\left(\alpha M + (1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n}\right)w = \\
\alpha M w + (1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n} w = \\
\alpha M^\top w + \frac{1-\alpha}{n} \mathbf{1}.
$$

Como os saltos aleatórios fazem parte do pagerank, chamaremos esta função de pageRankEsparso(M_sparse,niter,alpha).

**4.2** Implemente o método pageRankEsparso. Imprima o resultado de pageRankEsparso(X.T, niter=20,alpha=0.85).

In [141]:
def pageRankEsparso(A_sparse, niter=10, alpha=0.85):
    n = A_sparse.shape[0]
    w = np.ones((n,1))/n
    for i in range(niter):
        w = alpha * () + (1-alpha)/n *    
    return w

In [142]:
pageRankEsparso(X.T, niter=20)

array([[0.14180935],
       [0.20207835],
       [0.28796164],
       [0.36815065]])

**4.3** Vamos redefinir o método matrizDeTransicaoEsparsa para que ele não lançe uma exceção ao processar linhas cuja soma é igual a zero. Caso a soma da linha seja 0, a linha não deve ser alterada. Dica: Você pode calcular o vetor $d$ contendo a soma das linhas fazendo A_sparse.sum(axis=1) e substituir as entradas nulas de $d$ por qualquer constante. Porém, antes de usar $d$ para normalizar $A$, é preciso trasformá-lo em um numpy array fazendo d = d.A1

Imprima o resultado para $A_\textrm{new}$. Lembre-se de que você deve converter $A_\textrm{new}$ para o formato esparso $A_\textrm{new_sparse}$ primeiro.

In [187]:
def matrizDeTransicaoEsparsa(A_sparse):
    B = A_sparse.copy()
    d = B.sum(axis=1)

    B.data = B.data / 
    return B



**4.4** Vamos redefinir o método pageRankEsparso para que ele faça um tratamento especial das colunas cuja soma é zero. Note que as colunas nulas em $M$ correspondem aos *dangling nodes*. Para estes vértices, os saltos aleatórios totalizam 100% da probabilidade de transição e não apenas $1-\alpha$. Considere o seguinte exemplo **que deve ser generalizado** pelo método que você irá implementar.

Suponha que o segundo vértice é um *dangling node*. Neste caso, no lugar da matriz

$$
(1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n} =
\begin{pmatrix}
\frac{1-\alpha}{n} & \frac{1-\alpha}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\frac{1-\alpha}{n} & \frac{1-\alpha}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
\frac{1-\alpha}{n} & \frac{1-\alpha}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\end{pmatrix}
$$

deve ser utilizada a matriz

$$
R = 
\begin{pmatrix}
\frac{1-\alpha}{n} & \frac{1}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\frac{1-\alpha}{n} & \frac{1}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
\frac{1-\alpha}{n} & \frac{1}{n}& \frac{1-\alpha}{n} & \ldots & \frac{1-\alpha}{n} \\
\end{pmatrix} = 
(1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n} +
\begin{pmatrix}
0 & \frac{\alpha}{n}& 0 & \ldots & 0 \\
0 & \frac{\alpha}{n}& 0 & \ldots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & \frac{\alpha}{n}& 0 & \ldots & 0 \\
\end{pmatrix}
$$

Note que
$$Rw = (1-\alpha) \frac{\mathbf{1}\mathbf{1}^\top}{n} w + \frac{\alpha w_2}{n} \mathbf{1}.
$$

Imprima o resultado de pageRankEsparso para a matriz $A_\textrm{new_sparse}$. Lembre-se de que você precisa obter a matriz de transição $P_\textrm{new_sparse}$ a partir de $A_\textrm{new_sparse}$.

In [118]:
def pageRankEsparso(A_sparse, niter=10, alpha=0.85):
    n = A_sparse.shape[0]
    d = A_sparse.sum(axis=0).A1
    
    w = np.ones((n,1))/n
    for i in range(niter):
        w = alpha * () +  + 
    return w

In [119]:
pageRankEsparso(P_new_sparse.T,niter=10)

array([[0.12189213],
       [0.17370957],
       [0.24754394],
       [0.33496224],
       [0.12189213]])

**4.5** Execute o método pageRankEsparso para a matriz de adjacências do Wikipedia em Português, obtida através do notebook DecomposicaoEspectral. Retorne o título da página com maior pagerank. Atenção: o código abaixo assume que a matriz de adjacência $A$ foi salva no arquivo 'data/dbpedia/A.pkl' em formato CSC.

In [152]:
import pickle

PATH = 'data/dbpedia/'
A_wiki = pickle.load(open(PATH+'A.pkl', 'rb'))
index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))

names = {i: name for name, i in index_map.items()}

In [201]:
def show_ex(v):
    print('\n'.join(names[i] for i in np.abs(v.squeeze()).argsort()[-1:-20:-1]))

In [195]:
= matrizDeTransicaoEsparsa(A_wiki)
pr_wiki = pageRankEsparso(,niter=20,alpha=0.85)

In [202]:
show_ex(pr_wiki)

Anexo:Lista_de_asteroides
Brasil
Estados_Unidos
Quil\u00F3metro_quadrado
Densidade_populacional
Cintura_de_asteroides
Wikip\u00E9dia:Artigo_sobre_evento_futuro
Portugal
Ficheiro:Small-city-symbol.svg
Categoria:Desambigua\u00E7\u00E3o
Animalia
Censo_demogr\u00E1fico
Fran\u00E7a
Alemanha
Regi\u00F5es_da_Fran\u00E7a
Comuna_francesa
Departamentos_da_Fran\u00E7a
Plantae
Asteroide
